In [1]:
from ultralytics import YOLO
model = YOLO("dataset/best.pt",task="detect")

WARNING ⚠️ user config directory is not writeable, defaulting to '\tmp\Ultralytics'.


In [2]:
from collections import defaultdict

import cv2
import numpy as np

from ultralytics import YOLO

# Load the YOLOv8 model

# Open the video file
video_path = r"fainting_videos\vlc-record-2023-11-27-14h07m42s-SOKAK ORTASINDA BAYILMA ŞAKASI(Sosyal Deney)!!!.mp4-.mp4"
cap = cv2.VideoCapture(video_path)
writer = cv2.VideoWriter("test5.mp4",-1,30,(int(cap.get(3)),int(cap.get(4))),True)
# Store the track history
track_history = defaultdict(lambda: [])
frame_count = 0
frame_count_last = {}
frame_count_first = {}

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        try:    
        # Run YOLOv8 tracking on the frame, persisting tracks between frames
            results = model.track(frame, persist=True)

        # Get the boxes and track IDs
            boxes = results[0].boxes.xywh.cpu()
            track_ids = results[0].boxes.id.int().cpu().tolist()

        # Visualize the results on the frame
            annotated_frame = results[0].plot()
            if len(track_ids):
                for i,trk_id in enumerate(track_ids):
                    if trk_id not in frame_count_first:
                        frame_count_last[trk_id] = frame_count
                        frame_count_first[trk_id] = frame_count
                    if frame_count_last[trk_id]>frame_count_first[trk_id]:
                        frame_count_last[trk_id] = frame_count
                    else:
                        frame_count_first[trk_id] = frame_count
        # Plot the tracks
            for box, track_id in zip(boxes, track_ids):
                x, y, w, h = box
                track = track_history[track_id]
                track.append((float(x), float(y)))  # x, y center point
                if len(track) > 30:  # retain 90 tracks for 90 frames
                    track.pop(0)

            # Draw the tracking lines
                duration = (frame_count_first[track_id]-frame_count_last[track_id])/30
                points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
                cv2.putText(annotated_frame,f"duration: {round(duration)}",(int(x),int(y+10)),cv2.FONT_HERSHEY_COMPLEX,1,(230,230,230),1)

        # Display the annotated frame
            cv2.imshow("YOLOv8 Tracking", annotated_frame)
            writer.write(annotated_frame)

        # Break the loop if 'q' is pressed
            if cv2.waitKey(1) & 0xFF == ord("q"):
                break
        except (AttributeError):
            continue
        frame_count+=1
    else:
        # Break the loop if the end of the video is reached
        break
    

# Release the video capture object and close the display window
cap.release()
writer.release()
cv2.destroyAllWindows()


0: 384x640 5 kosan-insans, 1 yatan-insan, 48.8ms
Speed: 0.0ms preprocess, 48.8ms inference, 55.5ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 5 kosan-insans, 1 yatan-insan, 6.8ms
Speed: 0.0ms preprocess, 6.8ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 7 kosan-insans, 1 yatan-insan, 6.4ms
Speed: 3.5ms preprocess, 6.4ms inference, 4.3ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 7 kosan-insans, 1 yatan-insan, 7.2ms
Speed: 3.2ms preprocess, 7.2ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 8 kosan-insans, 1 yatan-insan, 7.6ms
Speed: 2.8ms preprocess, 7.6ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 7 kosan-insans, 1 yatan-insan, 8.9ms
Speed: 6.3ms preprocess, 8.9ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 7 kosan-insans, 1 yatan-insan, 8.3ms
Speed: 4.5ms preprocess, 8.3ms inference, 1.5ms postprocess per image at shape